# Import Python packages

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Import PyoD Packages & the methods

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

C:\Users\91845\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


# Import Metrics Packages

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

# Define data file and read X and y

In [4]:
mat_files_list= ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat', 
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']
mat_files_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [5]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc','ABOD','CBLOF','FB','HBOS','IForest','KNN','LOF','MCD','OCSVM','PCA']

roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

# Load mat file

In [6]:
from scipy.io import loadmat

In [7]:
data=loadmat("C:/Users/91845/Desktop/Jagruti/Anomaly Detection Project/Anamoly_detec_data/cardio.mat")
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [8]:
len(data)

5

In [9]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [10]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS',
            'ABOD','IFOREST','FEATUREBAGGING']

In [11]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

# Precision Time and Roc evolution tables creation

In [12]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


# Exploring All Mat Files

In [14]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_files_list:
   print("\n... Processing", mat_file, '...')
   mat = loadmat(os.path.join('C:/Users/91845/Desktop/Jagruti/Anomaly Detection Project/Anamoly_detec_data', mat_file))

   X = mat['X']
   y = mat['y'].ravel()
   outliers_fraction = np.count_nonzero(y) / len(y)
   outliers_percentage = round(outliers_fraction * 100, ndigits=4)

   # construct containers for saving results
   roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

   # 60% data for training and 40% for testing
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                       random_state=random_state)

   # standardizing data for processing
   X_train_norm, X_test_norm = standardizer(X_train, X_test)

   classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
       contamination=outliers_fraction),
       'Cluster-based Local Outlier Factor': CBLOF(
           contamination=outliers_fraction, check_estimator=False,
           random_state=random_state),
       'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                         random_state=random_state),
       'Histogram-base Outlier Detection (HBOS)': HBOS(
           contamination=outliers_fraction),
       'Isolation Forest': IForest(contamination=outliers_fraction,
                                   random_state=random_state),
       'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
       'Local Outlier Factor (LOF)': LOF(
           contamination=outliers_fraction),
       'Minimum Covariance Determinant (MCD)': MCD(
           contamination=outliers_fraction, random_state=random_state),
       'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
       'Principal Component Analysis (PCA)': PCA(
           contamination=outliers_fraction, random_state=random_state),
   }

   for clf_name, clf in classifiers.items():
       t0 = time()
       clf.fit(X_train_norm)
       test_scores = clf.decision_function(X_test_norm)
       t1 = time()
       duration = round(t1 - t0, ndigits=4)
       time_list.append(duration)

       roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
       prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

       print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
             'execution time: {duration}s'.format(
           clf_name=clf_name, roc=roc, prn=prn, duration=duration))

       roc_list.append(roc)
       prn_list.append(prn)

   temp_df = pd.DataFrame(time_list).transpose()
   temp_df.columns = df_columns
   time_df = pd.concat([time_df, temp_df], axis=0)

   temp_df = pd.DataFrame(roc_list).transpose()
   temp_df.columns = df_columns
   roc_df = pd.concat([roc_df, temp_df], axis=0)

   temp_df = pd.DataFrame(prn_list).transpose()
   temp_df.columns = df_columns
   prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 3.8313s
Cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643, execution time: 2.6925s
Feature Bagging ROC:0.7799, precision @ rank n:0.5, execution time: 0.6551s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 3.1463s
Isolation Forest ROC:0.8527, precision @ rank n:0.5714, execution time: 0.7839s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.2088s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.1574s


C:\Users\91845\Anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 1.1978s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.0317s
Principal Component Analysis (PCA) ROC:0.8, precision @ rank n:0.5, execution time: 0.0572s

... Processing cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.5763, precision @ rank n:0.1875, execution time: 0.4371s
Cluster-based Local Outlier Factor ROC:0.8221, precision @ rank n:0.4844, execution time: 0.2668s
Feature Bagging ROC:0.4879, precision @ rank n:0.1406, execution time: 0.9989s
Histogram-base Outlier Detection (HBOS) ROC:0.8453, precision @ rank n:0.4688, execution time: 0.008s
Isolation Forest ROC:0.9414, precision @ rank n:0.5, execution time: 0.4961s
K Nearest Neighbors (KNN) ROC:0.6959, precision @ rank n:0.2812, execution time: 0.1307s
Local Outlier Factor (LOF) ROC:0.4715, precision @ rank n:0.125, execution time: 0.0952s


C:\Users\91845\Anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8781, precision @ rank n:0.3906, execution time: 0.6316s
One-class SVM (OCSVM) ROC:0.9507, precision @ rank n:0.5938, execution time: 0.0727s
Principal Component Analysis (PCA) ROC:0.9638, precision @ rank n:0.6875, execution time: 0.0125s

... Processing glass.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7104, precision @ rank n:0.25, execution time: 0.0722s
Cluster-based Local Outlier Factor ROC:0.8506, precision @ rank n:0.25, execution time: 0.064s
Feature Bagging ROC:0.7043, precision @ rank n:0.25, execution time: 0.0407s
Histogram-base Outlier Detection (HBOS) ROC:0.6524, precision @ rank n:0.0, execution time: 0.0126s
Isolation Forest ROC:0.7195, precision @ rank n:0.25, execution time: 0.6078s
K Nearest Neighbors (KNN) ROC:0.7805, precision @ rank n:0.25, execution time: 0.0142s
Local Outlier Factor (LOF) ROC:0.7774, precision @ rank n:0.25, execution time: 0.002s
Minimum Covariance Determinant (MCD) ROC:0.7165, precision @ rank 

...

Minimum Covariance Determinant (MCD) ROC:0.863, precision @ rank n:0.3973, execution time: 4.1215s
One-class SVM (OCSVM) ROC:0.8417, precision @ rank n:0.3801, execution time: 4.201s
Principal Component Analysis (PCA) ROC:0.8396, precision @ rank n:0.3767, execution time: 0.1483s

... Processing musk.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.0809, precision @ rank n:0.0333, execution time: 2.3112s
Cluster-based Local Outlier Factor ROC:1.0, precision @ rank n:1.0, execution time: 0.5183s
Feature Bagging ROC:0.5228, precision @ rank n:0.1667, execution time: 13.0021s
Histogram-base Outlier Detection (HBOS) ROC:0.9999, precision @ rank n:0.9667, execution time: 0.0843s
Isolation Forest ROC:0.9996, precision @ rank n:0.9333, execution time: 1.6739s
K Nearest Neighbors (KNN) ROC:0.7348, precision @ rank n:0.2333, execution time: 1.9774s
Local Outlier Factor (LOF) ROC:0.5323, precision @ rank n:0.1333, execution time: 1.7007s
Minimum Covariance Determinant (MCD) ROC:1.0, precision @

C:\Users\91845\Anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.529726868394015 > -76.063390118997717). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\91845\Anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.112738611085312 > -76.438110891251171). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\91845\Anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.624467419142938 > -74.461731344350056). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\User

Minimum Covariance Determinant (MCD) ROC:0.99, precision @ rank n:0.7487, execution time: 18.638s
One-class SVM (OCSVM) ROC:0.9924, precision @ rank n:0.9584, execution time: 70.324s
Principal Component Analysis (PCA) ROC:0.9909, precision @ rank n:0.9555, execution time: 0.071s

... Processing vertebral.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.3429, precision @ rank n:0.0769, execution time: 0.082s
Cluster-based Local Outlier Factor ROC:0.4152, precision @ rank n:0.0769, execution time: 0.079s
Feature Bagging ROC:0.4467, precision @ rank n:0.0769, execution time: 0.0612s
Histogram-base Outlier Detection (HBOS) ROC:0.2892, precision @ rank n:0.0, execution time: 0.0s
Isolation Forest ROC:0.3475, precision @ rank n:0.0769, execution time: 0.5651s
K Nearest Neighbors (KNN) ROC:0.291, precision @ rank n:0.0, execution time: 0.0304s
Local Outlier Factor (LOF) ROC:0.4421, precision @ rank n:0.0769, execution time: 0.0164s
Minimum Covariance Determinant (MCD) ROC:0.3698, precision @

C:\Users\91845\Anaconda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-23.044572678546199 > -40.825433830407448). You may want to try with a higher value of support_fraction (current value: 0.528).
  RuntimeWarning)



... Processing vowels.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.9683, precision @ rank n:0.45, execution time: 0.4921s
Cluster-based Local Outlier Factor ROC:0.8891, precision @ rank n:0.25, execution time: 0.1489s
Feature Bagging ROC:0.9422, precision @ rank n:0.3, execution time: 0.3579s
Histogram-base Outlier Detection (HBOS) ROC:0.7339, precision @ rank n:0.15, execution time: 0.0105s
Isolation Forest ROC:0.7897, precision @ rank n:0.2, execution time: 0.6986s
K Nearest Neighbors (KNN) ROC:0.9782, precision @ rank n:0.55, execution time: 0.1305s
Local Outlier Factor (LOF) ROC:0.9345, precision @ rank n:0.3, execution time: 0.0712s
Minimum Covariance Determinant (MCD) ROC:0.6465, precision @ rank n:0.0, execution time: 1.0349s
One-class SVM (OCSVM) ROC:0.8225, precision @ rank n:0.3, execution time: 0.0361s
Principal Component Analysis (PCA) ROC:0.626, precision @ rank n:0.15, execution time: 0.0s

... Processing wbc.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.9005, p

In [15]:
roc_df 

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.8
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8781,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,pima,768,8,34.8958,0.719,0.6866,0.6524,0.7003,0.716,0.7426,0.6623,0.7251,0.6633,0.6767
0,satellite,6435,36,31.6395,0.5654,0.764,0.5251,0.7743,0.6868,0.67,0.5272,0.7935,0.6613,0.595


In [16]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,pima,768,8,34.8958,0.6218,0.5714,0.5378,0.563,0.6218,0.6218,0.5378,0.605,0.563,0.6134
0,satellite,6435,36,31.6395,0.395,0.611,0.3723,0.5716,0.5668,0.4881,0.3675,0.6778,0.5406,0.4881


In [17]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,3.8313,2.6925,0.6551,3.1463,0.7839,0.2088,0.1574,1.1978,0.0317,0.0572
0,cardio,1831,21,9.6122,0.4371,0.2668,0.9989,0.008,0.4961,0.1307,0.0952,0.6316,0.0727,0.0125
0,glass,214,9,4.2056,0.0722,0.064,0.0407,0.0126,0.6078,0.0142,0.002,0.0896,0.0136,0.0206
0,ionosphere,351,33,35.8974,0.1511,0.101,0.0688,0.011,0.5472,0.0163,0.0117,0.0756,0.008,0
0,letter,1600,32,6.25,0.4129,0.1835,0.9447,0.0216,0.7034,0.3055,0.1493,1.3937,0.0788,0.0101
0,lympho,148,18,4.0541,0.0413,0.0639,0.0408,0,0.5015,0.0098,0.008,0.0778,0.0061,0.0102
0,mnist,7603,100,9.2069,9.6864,1.3581,65.6768,0.0856,3.2008,8.4605,7.1885,4.1215,4.201,0.1483
0,musk,3062,166,3.1679,2.3112,0.5183,13.0021,0.0843,1.6739,1.9774,1.7007,20.0351,1.1088,0.2553
0,pima,768,8,34.8958,0.1709,0.1277,0.1618,0,0.5231,0.0317,0.0111,0.0637,0.0161,0.002
0,satellite,6435,36,31.6395,2.7293,0.4922,8.2665,0.049,1.0253,1.1999,0.9435,3.0675,1.3765,0.0311
